# XG Boost Classifier

## 1. Loading the dataset

In [1]:
# 1.1. Launch commands to automatically reload modules
%load_ext autoreload
%autoreload 2

In [2]:
# 1.2. Import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

In [3]:
# 1.3. Load the training, validation and test datasets into a dataframes 
# Load the Parquet files into DataFrames
X_train = pd.read_parquet('../data/processed/X_train.parquet')
X_val = pd.read_parquet('../data/processed/X_val.parquet')
y_train = pd.read_parquet('../data/processed/y_train.parquet')
y_val = pd.read_parquet('../data/processed/y_val.parquet')
X_test = pd.read_parquet('../data/processed/X_test_scaled.parquet')

In [4]:
# taking out the player id out of the test data
player_id_test = X_test.pop('player_id')

In [5]:
y_val = y_val.iloc[:, 0]  # Convert the single column DataFrame to a Series
y_val.shape

(9324,)

In [6]:
y_train = y_train.iloc[:, 0]  # Convert the single column DataFrame to a Series
y_train.shape

(37294,)

## 3. XGBoost - Modelling

In [7]:
#3.1 Import XGBoost and instantiate the model
import xgboost as xgb

# Initialize the GB model
xg1 = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# 3.2 Train the model and save the predicted values
# Fit the model on the training data
xg1.fit(X_train, y_train)


In [ ]:
# Select the probability for the positive class (class 1) - GB with out feature importance
y_train_probs_xg1 = xg1.predict_proba(X_train)[:, 1]
y_val_probs_xg1 = xg1.predict_proba(X_val)[:, 1]


In [ ]:
# 3.3 To plot ROC curve and the AUC metric - for comparison of model performances
#To plot ROC curve and the AUC metric
from sklearn import metrics

# Set up plotting area
plt.figure(0).clf()

# XG Boost- Train
fpr, tpr, _ = metrics.roc_curve(y_train, y_train_probs_xg1)
auc = round(metrics.roc_auc_score(y_train, y_train_probs_xg1), 4)
plt.plot(fpr, tpr, label="XG Boost Train, AUC="+str(auc))

# XG Boost - Validation
fpr, tpr, _ = metrics.roc_curve(y_val, y_val_probs_xg1)
auc = round(metrics.roc_auc_score(y_val, y_val_probs_xg1), 4)
plt.plot(fpr, tpr, label="XG Boost Val, AUC="+str(auc))

# Add legend
plt.legend()

# Show the plot
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# 3.4 Save the probability of the test data
y_test_probs_xg1 = xg1.predict_proba(X_test)[:, 1]


In [ ]:
# 3.5 add the 'player_id'column  into the prediction probability
# Convert y_test_prob_rf3 (which is a NumPy array) to a DataFrame
y_test_probs_df = pd.DataFrame(y_test_probs_xg1, columns=['drafted'])

# Concatenate player_id_test and y_test_prob_df along the columns
Results = pd.concat([player_id_test, y_test_probs_df], axis=1)

In [ ]:
Results

In [ ]:
#3.6 save the probability as CSV file

# Convert to DataFrame and save
pd.DataFrame(Results).to_csv('../data/external/XGBoost_Results.csv', index=False)

#3.7 save the model
# Import dump from joblib
from joblib import dump

# 3.12 Save the model and call the files respectively GB.joblib 
dump(gb1, '../models/XGBoost.joblib')
